<a href="https://colab.research.google.com/github/Buzon-coder/QRT-Electricity-price/blob/main/QRT_Electricity_Price2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#from google.colab import files
#uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
import numpy as np

In [39]:
X_test = pd.read_csv("/content/drive/MyDrive/Colab data/X_test_final.csv")
X_train = pd.read_csv("/content/drive/MyDrive/Colab data/X_train_NHkHMNU.csv")
y_train = pd.read_csv("/content/drive/MyDrive/Colab data/y_train_ZAN5mwg.csv")

In [40]:
common_id = set(X_train["ID"]) & set(X_test["ID"])
print("Nombre d'ID' en commun entre TRAIN et TEST:",len(common_id))

common_days = set(X_train["DAY_ID"]) & set(X_test["DAY_ID"])
print("Nombre de jours en commun entre TRAIN et TEST:",len(common_days))

Nombre d'ID' en commun entre TRAIN et TEST: 0
Nombre de jours en commun entre TRAIN et TEST: 0


In [41]:
# je trie X_train par DAY_ID croissant (donc par dates) et j'aligne les ID de y_train avec ceux de DAY_ID

X_train = X_train.sort_values(["DAY_ID", "COUNTRY"]).reset_index(drop=True)
y_train = y_train.set_index("ID").loc[X_train["ID"]].reset_index()
assert all(X_train["ID"].values == y_train["ID"].values)
print("Les ID sont bien alignés avec X_train")

Les ID sont bien alignés avec X_train


In [42]:
#print(X_train.head())

J'encode les features de X_train qui ne sont pas des int ou des float (en l'occurence, seul "COUNTRY" n'est pas un float ou un int, donc c'est la seule colonne que j'encode)

In [43]:
##j'encode la colonne COUNTRY en 2 colonnes : COUNTRY_FR et COUNTRY_DE, et je supprime la colonne COUNTRY
X_train_encode = X_train.copy()
X_train_encode["COUNTRY_FR"] = (X_train_encode["COUNTRY"] == "FR").astype(int)
X_train_encode["COUNTRY_DE"] = (X_train_encode["COUNTRY"] == "DE").astype(int)
X_train_encode = X_train_encode.drop(columns = ["COUNTRY"])
#print(X_train_encode.head())

On regarde maintenant les corrélations (en valeur absolue) entre les features et les TARGET

In [44]:
corrs = X_train_encode.corrwith(y_train["TARGET"], method = 'spearman') # Remarque : la fonction corrwith calcule la corrélation entre les features de X_train et les valeurs TARGET de y_train ligne par ligne
                                                                        # Remarque 2 : on utilise la corrélation de Spearman car c'est la corrélation de l'énoncé
corrs_abs = corrs.abs().sort_values(ascending=False) # on regarde les valeurs absolues des corrélations des features avec la TARGET. En effet, une corrélation négative mais avec une valeur absolue élevée est aussi une corrélation forte, c'est simplement
                                                     # qu'elle agit dans le sens inverse de l'évolution du prix du futures (qd feature augmente, cible diminue, ou l'inverse). exemple : si augmentation du vent, prix diminue car on a + d'électricité
#print(corrs_abs.head(20))
#print(corrs_abs.tail(18))

J'obtiens que les features les plus corrélées (en valeur absolue) de X_train et avec les TARGET (de y_train) sont par ordre décroissant : DE_NET_IMPORT, DE_NET_EXPORT, DE_WINDPOW, DE_RESIDUAL_LOAD, FR_WINDPOW,DE_HYDRO, DE_GAS, CARBON_RET, DE_WIND, DE_COAL, GAS_RET, FR_HYDRO,FR_WIND, DE_CONSUMPTION, FR_RAIN, FR_DE_EXCHANGE, DE_FR_EXCHANGE, FR_COAL, FR_TEMP, DE_LIGNITE.


De même, j'obtiens les features les moins corrélées (en valeur absolue) de X_train avec les TARGET sont (de la plus corrélées à la moins corrélée) : DE_RAIN, DE_SOLAR, FR_SOLAR, FR_CONSUMPTION, ID, COUNTRY_FR, COUNTRY_DE, DE_TEMP, FR_RESIDUAL_LOAD, COAL_RET, FR_GAS, FR_NET_EXPORT, FR_NET_IMPORT, DE_NUCLEAR, DAY_ID, FR_NUCLEAR.

Regarder les corrélations permet de comprendre quelles features influencent TARGET, et permet d'interpréter phyisquement le modèle. Ces features représentent les effets dominants.

Nettoyage des données

In [45]:
# On regarde si les features de X_train_encode contiennent des NaN dans le pays qui leur est associé.
# Tout d'abord, on regarde quelles features contiennent des NaN

print(X_train_encode.isna().sum())

ID                    0
DAY_ID                0
DE_CONSUMPTION        0
FR_CONSUMPTION        0
DE_FR_EXCHANGE       25
FR_DE_EXCHANGE       25
DE_NET_EXPORT       124
FR_NET_EXPORT        70
DE_NET_IMPORT       124
FR_NET_IMPORT        70
DE_GAS                0
FR_GAS                0
DE_COAL               0
FR_COAL               0
DE_HYDRO              0
FR_HYDRO              0
DE_NUCLEAR            0
FR_NUCLEAR            0
DE_SOLAR              0
FR_SOLAR              0
DE_WINDPOW            0
FR_WINDPOW            0
DE_LIGNITE            0
DE_RESIDUAL_LOAD      0
FR_RESIDUAL_LOAD      0
DE_RAIN              94
FR_RAIN              94
DE_WIND              94
FR_WIND              94
DE_TEMP              94
FR_TEMP              94
GAS_RET               0
COAL_RET              0
CARBON_RET            0
COUNTRY_FR            0
COUNTRY_DE            0
dtype: int64


--> on voit que la colonne FR_GAS et DE_GAS contiennent 2 fois les mêmes valeurs pour chaque DAY_ID.
En fait, c'est le cas pour toutes mes variables de mesure de production d'énergie et de mesures météorologiques.

Il semblerait que les variables de mesure de production d'énergie ont tout le temps la même valeur pour un même jour, peu importe s'il s'agit d'une ligne Allemagne ou d'une ligne France. Remarque : elles ne contiennent aucun NaN.

De même, il semblerait que les variables de mesure météorologiques ont tout le temps la même valeur pour un même jour, peu importe s'il s'agit d'une ligne Allemagne ou d'une ligne France. Remarque : elles ne sont pas complètes (elles contiennent des NaN, et d'ailleurs ces NaN sont situés aux mêmes lignes)

Je veux vérifier que pour toutes les lignes qui ont le même DAY_ID, les valeurs de DE_RAIN sont les mêmes.
Puis je ferai la même chose avec toutes mes autres variables météorologiques, puis avec toutes mes variables de mesure de production d'énergie

Afin de vérifier sur mes données TRAIN et TEST si les valeurs des variables météorologiques et de mesure de production sont les mêmes pour un même jour (peu importe s'il s'agit de la ligne Allemagne ou France), je concatène X_train et X_test, et je fais mes vérifications là-dessus

In [46]:
X_all = pd.concat([X_train, X_test], ignore_index=True).sort_values(["COUNTRY", "DAY_ID"])

In [47]:
features = ["DE_RAIN", "DE_WIND", "DE_TEMP", "FR_RAIN", "FR_WIND", "FR_TEMP", "DE_GAS", "DE_COAL", "DE_HYDRO", "DE_NUCLEAR", "DE_SOLAR", "DE_WINDPOW",
            "FR_GAS", "FR_COAL", "FR_HYDRO", "FR_NUCLEAR", "FR_SOLAR", "FR_WINDPOW", "DE_CONSUMPTION", "DE_FR_EXCHANGE", "DE_NET_EXPORT", "DE_NET_IMPORT",
            "DE_RESIDUAL_LOAD", "FR_CONSUMPTION", "FR_DE_EXCHANGE", "FR_NET_EXPORT", "FR_NET_IMPORT", "FR_RESIDUAL_LOAD", "CARBON_RET", "GAS_RET", "COAL_RET"]
results = {}
for feature in features:
  variances = X_all.groupby("DAY_ID")[feature].nunique()
  identiques = (variances <=1).all()
  results[feature] = identiques

results_df = pd.DataFrame.from_dict(results, orient="index", columns=["Constante_par_jour"])
print(results_df)



                  Constante_par_jour
DE_RAIN                         True
DE_WIND                         True
DE_TEMP                         True
FR_RAIN                         True
FR_WIND                         True
FR_TEMP                         True
DE_GAS                          True
DE_COAL                         True
DE_HYDRO                        True
DE_NUCLEAR                      True
DE_SOLAR                        True
DE_WINDPOW                      True
FR_GAS                          True
FR_COAL                         True
FR_HYDRO                        True
FR_NUCLEAR                      True
FR_SOLAR                        True
FR_WINDPOW                      True
DE_CONSUMPTION                  True
DE_FR_EXCHANGE                  True
DE_NET_EXPORT                   True
DE_NET_IMPORT                   True
DE_RESIDUAL_LOAD                True
FR_CONSUMPTION                  True
FR_DE_EXCHANGE                  True
FR_NET_EXPORT                   True
F

J'ai vérifié que toutes mes features (excepté ID, DAY_ID et COUNTRY) dans X_train et X_test ont les mêmes valeurs pour un même DAY_ID (peu importe si on est à une ligne Allemagne ou France).

Maintenant, je vais compléter les NaN pour :
- les variables météorologiques FR_TEMP, DE_WIND, FR_WIND, DE_TEMP, DE_RAIN, FR_RAIN, en interpolant temporellement.
- les variables de mesure de production DE_NET_IMPORT, DE_NET_EXPORT, FR_NET_IMPORT, FR_NET_EXPORT, DE_FR_EXCHANGE, FR_DE_EXCHANGE. L'idée est de remplir les NaN avec une interpolation linéaire locale pour les trous courts, avec une moyenne mobile (rolling mean) pour lisser les périodes plus longues, et avec une propagation avant/après pour combler les bords éventuels.

In [48]:
# imputation des NaN de mes variables météorologiques par interpolation temporelle
cols_meteo = ["FR_TEMP", "DE_WIND", "FR_WIND", "DE_TEMP", "DE_RAIN", "FR_RAIN"]

meteo_by_day = X_all.sort_values("DAY_ID").groupby("DAY_ID")[cols_meteo].mean() # on prend la moyenne des 2 valeurs d'une feature le même jour (les 2 valeurs sont égales sauf s'il y
                                                                                # a une ligne avec un NaN et l'autre avec une valeur), ça renvoie un dataset indexé sur les DAY_ID croissants
# print(meteo_by_day)

meteo_interp = (meteo_by_day.interpolate(methode='linear', limit_direction='both')) # interpolation temporelle linéaire
X_all = X_all.drop(columns=cols_meteo).merge(meteo_interp.reset_index(), on="DAY_ID", how="left")
# print(X_all.head())

# print(X_all.isna().sum()) #--> j'ai bien imputé les NaN de mes colonnes météo


In [49]:
# imputation des NaN de mes variables de mesure de production

cols_usage = [
    "DE_NET_IMPORT", "DE_NET_EXPORT",
    "FR_NET_IMPORT", "FR_NET_EXPORT",
    "DE_FR_EXCHANGE", "FR_DE_EXCHANGE"
]

usage_by_day = X_all.sort_values("DAY_ID").groupby("DAY_ID")[cols_usage].mean()
# print(usage_by_day)
usage_interp = usage_by_day.copy()
# imputation des valeurs NaN dans usage_by_day :
  # interpolation linéaire locale pour les trous courts
usage_interp = usage_interp.interpolate(method="linear", limit=3, limit_direction="both")
  # moyenne mobile pour lisser les périodes les plus longues
usage_interp = usage_interp.fillna(usage_interp.rolling(window=3, min_periods=1, center=True).mean())
  # propagation avant après pour combler les bords éventuels
usage_interp = usage_interp.ffill().bfill()

X_all = X_all.drop(columns=cols_usage).merge(usage_interp.reset_index(), on="DAY_ID", how="left")

# print(X_all.isna().sum())


J'ai mon X_all complet (sans NaN), maintenant je vais OneHotEncoder ma variable catégorielle COUNTRY

In [50]:
X_all_encode = X_all.copy()

X_all_encode["COUNTRY_DE"] = (X_all_encode["COUNTRY"] == "DE").astype(int)
X_all_encode["COUNTRY_FR"] = (X_all_encode["COUNTRY"] == "FR").astype(int)

X_all_encode = X_all_encode.drop(columns = "COUNTRY")

# print(X_all_encode.head())

J'ai mon X_all complet (sans NaN) et avec que des variables qui sont des int ou des float grâce au OneHotEncoding.

Désormais, je veux créer des lags sur mes variables. Pour choisir les lags pour chaque variable, je dois regarder pour chaque variable pour quels lags la corrélation croisée est la plus forte.
Dans un premier temps, je fusionne mes TARGET avec mon dataset X_all, et j'appelle le tout **train_all**.

In [51]:
train_all = X_all_encode.merge(y_train[["ID", "TARGET"]], on="ID", how="left")
#print(train_all.head())

In [52]:
# Objectif de cette cellule de code : renvoyer, pour chaque feature et chaque pays, les top_k lags (top_k_xcorr_all_features) par Spearman jusqu'à max_lag

def _daily_series(df, col):
  # renvoie une série indexée par DAY_ID (comme les valeurs sont les mêmes pour le DAY_ID, ça se fait facilement en agregeant par mean)
  s = (df.groupby("DAY_ID")[col].mean().sort_index())
  return s


def _daily_target_by_country(df, country_code, target_col="TARGET"):
  # renvoie la TARGET du pays demandé, indexée par DAY_ID, donc on filtre par pays
  if country_code == "DE":
    d = df[df["COUNTRY_DE"] == 1]
  elif country_code == "FR":
    d = df[df["COUNTRY_FR"] == 1]
  y = (d.groupby("DAY_ID")[target_col].mean().sort_index())
  return y


def xcorr_feature_target(df, feature, country_code, target_col="TARGET", max_lag=5, method="spearman", min_overlap=20):
  # Calcule la corrélation entre Target_country(t) et Feature (t-k) pour k=0..max_lag.
  # Retourne un DataFrame : lag, corr, abs_corr, n_overlap
  x = _daily_series(df, feature) # série quotidienne unique du feature
  y = _daily_target_by_country(df, country_code, target_col) # série quotidienne unique de la target par pays
  idx = x.index.intersection(y.index) # aligne sur l'intersection des jours
  x = x.loc[idx]
  y = y.loc[idx]

  rows=[]
  for k in range(0, max_lag+1):
    x_lag=x.shift(k)
    aligned = pd.concat([y, x_lag], axis=1, keys=["y", "x_lag"]).dropna() # on concatène les features laggées et la target
    n = len(aligned)
    if n < min_overlap:
      rows.append({"lag": k, "corr": np.nan, "abs_corr": np.nan, "n_overlap": n})
      continue
    corr = aligned["y"].corr(aligned["x_lag"], method=method)
    rows.append({"lag": k, "corr": corr, "abs_corr": abs(corr), "n_overlap": n})

  out=pd.DataFrame(rows)
  out["feature"]=feature
  out["country"]=country_code
  return out[["country", "feature", "lag", "corr", "abs_corr", "n_overlap"]]



def top_k_xcorr_all_features(df, features, countries=("DE","FR"), target_col="TARGET", max_lag=30, top_k=3, method="spearman", min_overlap=20):
  # Pour chaque (pays, feature), calcule la corrélation sur les lags (entre 0 et max_lag), et renvoie les top_k par corrélation

  all_rows=[]
  for c in countries:
    for f in features:
      res = xcorr_feature_target(df, feature=f, country_code=c, target_col=target_col, max_lag=max_lag, method=method, min_overlap=min_overlap)
      # garde les top_k lags par |ρ|
      res = res.sort_values("abs_corr", ascending=False).head(top_k)
      all_rows.append(res)

  out = pd.concat(all_rows, ignore_index=True)
  out = out.sort_values(["country", "abs_corr", "feature"], ascending=[True, False, True])
  return out




In [53]:
# je teste mon code

features = ["DE_RAIN", "DE_WIND", "DE_TEMP", "FR_RAIN", "FR_WIND", "FR_TEMP", "DE_GAS", "DE_COAL", "DE_HYDRO", "DE_NUCLEAR", "DE_SOLAR", "DE_WINDPOW",
            "FR_GAS", "FR_COAL", "FR_HYDRO", "FR_NUCLEAR", "FR_SOLAR", "FR_WINDPOW", "DE_CONSUMPTION", "DE_FR_EXCHANGE", "DE_NET_EXPORT", "DE_NET_IMPORT",
            "DE_RESIDUAL_LOAD", "FR_CONSUMPTION", "FR_DE_EXCHANGE", "FR_NET_EXPORT", "FR_NET_IMPORT", "FR_RESIDUAL_LOAD", "CARBON_RET", "GAS_RET", "COAL_RET"]
print(len(features))
res = top_k_xcorr_all_features(train_all, features, countries=("DE","FR"), target_col="TARGET", max_lag=5, top_k=3, method="spearman", min_overlap=20)
#print(res.shape)
print(res.head(10))


subset = res[(res["country"] == "DE") & (res["feature"] == "DE_GAS")] \
            .sort_values("abs_corr", ascending=False) \
            [["lag", "corr", "abs_corr", "n_overlap"]]

print(subset)


31
   country           feature  lag      corr  abs_corr  n_overlap
66      DE  DE_RESIDUAL_LOAD    0  0.324335  0.324335        643
60      DE     DE_NET_EXPORT    0 -0.306204  0.306204        643
63      DE     DE_NET_IMPORT    0  0.306204  0.306204        643
33      DE        DE_WINDPOW    0 -0.300933  0.300933        643
18      DE            DE_GAS    0  0.253410  0.253410        643
24      DE          DE_HYDRO    0  0.217900  0.217900        643
51      DE        FR_WINDPOW    0 -0.199820  0.199820        643
3       DE           DE_WIND    0 -0.146598  0.146598        643
21      DE           DE_COAL    0  0.142054  0.142054        643
25      DE          DE_HYDRO    5 -0.117970  0.117970        640
    lag      corr  abs_corr  n_overlap
18    0  0.253410  0.253410        643
19    3 -0.077079  0.077079        641
20    4 -0.016348  0.016348        640


# Création du dataset avec les lags (**train_all_lag**)

- Pour chaque couple (pays, feature), on prend le lag max trouvé dans res
- On crée les colonnes t-1 à t-L : FEATURE_COUNTRY_1, …, FEATURE_COUNTRY_L
- On masque ces colonnes (met NaN) sur les lignes qui ne correspondent pas au pays

In [54]:
def build_maxlag_map(res_df):
  """ A partir de 'res' (top_k par (country, feature)), construit un dict { (country, feature) : max_lag_trouve }."""
  # on ignore les lags NaN au cas où
  valid = res_df.dropna(subset=["lag"])
  maxlag = valid.groupby(["country", "feature"])["lag"].max()
  return maxlag.to_dict()

def add_lag_columns_for_pair(df, feature, country_code, max_lag, day_col="DAY_ID"):
  """Ajoute au DataFrame 'df' les colonnes laggées pour un couple (feature, country_code),
     depuis t-1 jusqu'à t-max_lag. Masque ensuite les lignes du pays non concerné en NaN.
     Noms de colonnes créés: f"{feature}_{country_code}_{k}" pour k=1..max_lag"""

  if max_lag <= 0:
    return df
  daily = df.groupby(day_col)[feature].mean().sort_index()
  # On construit un petit DataFrame de lags
  lag_cols = {}
  for k in range(1, max_lag + 1):
    lag_cols[f"{feature}_{country_code}_{k}"] = daily.shift(k)
  lag_df = pd.DataFrame(lag_cols, index=daily.index).reset_index()

  # On fusionne sur DAY_ID (donc on fusionne même sur les lignes qui ne vérifient pas le country_code,
  # mais c'est pas grave car on remplira ces lignes par des NaN plus tard)
  df = df.merge(lag_df, on=day_col, how="left")

  # On remplit les lignes dont le pays n'est pas country_code par NaN

  mask_col = "COUNTRY_DE" if country_code == "DE" else "COUNTRY_FR"
  new_cols = [f"{feature}_{country_code}_{k}" for k in range(1, max_lag + 1)]
  df.loc[df[mask_col] != 1, new_cols] = np.nan

  return df



def build_train_all_lag(train_all, res, day_col="DAY_ID"):
  """Construit train_all_lag en ajoutant toutes les colonnes laggées
     pour chaque (pays, feature) jusqu'au lag maximum trouvé dans 'res'."""

  train_all_lag = train_all.copy()
  maxlag_map = build_maxlag_map(res)

  for (country_code, feature), L in maxlag_map.items():
    if feature not in train_all_lag.columns:
      print(f"[AVERTISSEMENT] Colonne '{feature}' absente, sautée.")
      continue
    train_all_lag = add_lag_columns_for_pair(train_all_lag, feature=feature, country_code=country_code, max_lag=L, day_col=day_col)

  return train_all_lag

In [58]:
# je teste mon code

train_all_lag = build_train_all_lag(train_all, res, day_col="DAY_ID")
print([c for c in train_all_lag.columns if c.startswith("DE_GAS_DE_")][:10])
print(train_all_lag.shape)
print(train_all_lag.head())

['DE_GAS_DE_1', 'DE_GAS_DE_2', 'DE_GAS_DE_3', 'DE_GAS_DE_4']
(2148, 309)
    ID  DAY_ID  DE_CONSUMPTION  FR_CONSUMPTION    DE_GAS    FR_GAS   DE_COAL  \
0  111       2       -0.068972       -0.667390  1.228079  0.458302 -0.247704   
1  800       3       -0.134670       -0.834564  1.588542  0.069297 -0.635452   
2  410       4        0.944117        0.203547  0.451085  0.865857 -0.752504   
3  831       5       -0.297850       -0.470371  1.059828  0.528273 -0.072071   
4  562       6        0.120788       -0.744840  1.524895 -0.395402 -0.079164   

    FR_COAL  DE_HYDRO  FR_HYDRO  ...  FR_TEMP_FR_3  FR_TEMP_FR_4  \
0 -0.766904  1.785758 -0.930172  ...           NaN           NaN   
1 -0.718729  1.994144 -0.383690  ...           NaN           NaN   
2 -0.782998 -0.291237 -0.170210  ...           NaN           NaN   
3 -0.766063  1.275857 -0.398178  ...           NaN           NaN   
4 -0.794950  2.327561  0.074597  ...           NaN           NaN   

   FR_WIND_FR_1  FR_WIND_FR_2  FR_WIN

Désormais mon train_all_lag contient 309 colonnes.

# Ajout des valeurs de prédiction (des leads bruités) dans mon dataset

Mais je peux considérer que les valeurs des futures au jour t dépendent des prédictions météorologiques des jours suivants non? Or les personnes chargées de spéculer sur les valeurs de ces futures disposent de modèles puissants pour anticiper les prédictions météorologiques des jours suivants. Mais moi je ne dispose que des mesures météorologiques des jours précédents et des jours suivants, donc comme je dois estimer ce que prédirait un trader pour les valeurs des futures des jours suivants, je peux simuler que je dispose de modèles météorologiques puissants en utilisant les données météorologiques des jours suivants que je bruite afin de faire des prédictions non ?

Que penses-tu de ce raisonnement ?